In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import sample
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
import random
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
import json
import gzip

In [2]:
raw_data = pd.read_csv("Data/Digital_Music.csv", names=['Item', 'User', 'Rating', 'Timestamp'])
data = raw_data.sample(n=30000)
data.drop(['Timestamp'], axis=1,inplace=True)
data = data[['User', 'Item', 'Rating']]

In [3]:
data

,User,Item,Rating
404882,AKVKVEQBSHWJ5,B00IIGL8FE,5.0
893007,A11649NHUSS5AV,B001NYO4R2,4.0
606649,A26A64X86VL1R4,B00004Z05F,5.0
279623,A3ECR56OXHZTFQ,B007JCP8NG,5.0
828961,A3AIO4KGWB5RSD,B0019E7GLW,5.0
...,...,...,...
1313023,A4NWH8W0V2H6M,B00HKVVMKO,4.0
1182119,AMAC2XRPTF2HW,B00ANRLAM2,5.0
884045,A2GT88E6M3FVB0,B000WQQULU,5.0
974295,A3GI590E91VG4P,B003RJ2LDU,5.0


In [4]:
new_df=data.head(10000)
ratings_matrix = new_df.pivot_table(values='Rating', index='User', columns='Item', fill_value=0)
ratings_matrix.head()

Item,0001527134,0006920055,1567698530,1570195641,1570586004,1785871269,1932192077,1937985202,3426958910,4901000616,...,B01HGB5USC,B01HGBNMJQ,B01HGWUXOM,B01HHQSIDA,B01HI9B8T2,B01HIUWHBY,B01HJ44F0U,B01HJ8ZXRK,B01HJ91HEC,B01HJ91MTW
User,,,,,,,,,,,,,,,,,,,,,
A0072041HVZ3465DXUOR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0257999SZ856J5VEUO7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0611551KU79JNSC7JJV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A101A5HAADYZMO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A102AR4TWGCWJD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = ratings_matrix.T
X1 = X
X.head()

User,A0072041HVZ3465DXUOR,A0257999SZ856J5VEUO7,A0611551KU79JNSC7JJV,A101A5HAADYZMO,A102AR4TWGCWJD,A1046K5YQC0BEL,A104LJG6U21W05,A105C8QAF4P5PT,A105SWS184BTQH,A105YBEKJF0L2H,...,AZVMSD72W3BBT,AZVSA9KOJUKG3,AZVSVPI8LA2BM,AZWEOW42H23BR,AZWLFDFPKLYAT,AZWULMAOSR7IR,AZX4W3CS3CB23,AZY1HUEW2XZ6F,AZY33L9MH0CKK,AZY4EXS7GB265
Item,,,,,,,,,,,,,,,,,,,,,
0001527134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0006920055,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1567698530,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
1570195641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1570586004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#Decomposing the Matrix

SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(8532, 10)

In [7]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(8532, 8532)

In [8]:
i = X.index[random.randint(0, np.size(correlation_matrix, 0))]

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

5215

In [9]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(8532,)

In [18]:
Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:25]

['B000003CI2',
 'B000026GO2',
 'B00004BZ1H',
 'B0000564WK',
 'B00005LNPX',
 'B000091NO4',
 'B000254G94',
 'B0002W1B22',
 'B0006LINKC',
 'B0007D08RA',
 'B0007D69PK',
 'B0007TXVI2',
 'B0007ZFZVM',
 'B000AOOXMC',
 'B000AQH8TK',
 'B000BRFRFK',
 'B000BYKUOQ',
 'B000CF4U22',
 'B000E26544',
 'B000FM9MIY',
 'B000FX4YCC',
 'B000GDYY08',
 'B000K5Y0ME',
 'B000K8PQX8',
 'B000KE3YL8']

In [19]:
def parse(path):
  g = gzip.open(path, 'rt', encoding='utf-8')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

review_data = getDF('Data/meta_Digital_Music.json.gz')
data_set = review_data[:30000]
data_set

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,[],,[],,Master Collection Volume One,"[B000002UEN, B000008LD5, B01J804JKE, 747403435...",[],,John Michael Talbot,[],"58,291 in CDs & Vinyl (","[B000002UEN, B000008LD5, 7474034352, B000008LD...","<img src=""https://images-na.ssl-images-amazon....",,,$18.99,0001377647,NaN
1,[],,[],,Hymns Collection: Hymns 1 &amp; 2,"[5558154950, B00014K5V4]",[],,Second Chapter of Acts,[],"93,164 in CDs & Vinyl (","[B000008KJ3, B000008KJ0, 5558154950, B000UN8KZ...","<img src=""https://images-na.ssl-images-amazon....",,,,0001529145,NaN
2,[],,[],,Early Works - Don Francisco,"[B00004RC05, B003H8F4NA, B003ZFVHPO, B003JMP1Z...",[],,Don Francisco,[],"875,825 in CDs & Vinyl (","[B003H8F4NA, B003ZFVHPO, B003JMP1ZK, B00004RC0...","<img src=""https://images-na.ssl-images-amazon....",,,,0001527134,NaN
3,[],,[],,So You Wanna Go Back to Egypt,"[B0000275QQ, 0001393774, 0001388312, B0016CP2G...",[],,Keith Green,[],"203,263 in CDs & Vinyl (","[B00000I7JO, B0016CP2GS, 0001393774, B0000275Q...","<img src=""https://images-na.ssl-images-amazon....",,,$13.01,0001388703,NaN
4,[],,[1. Losing Game 2. I Can't Wait 3. Didn't He S...,,Early Works - Dallas Holm,"[B0002N4JP2, 0760131694, B00002EQ79, B00150K8J...",[],,Dallas Holm,[],"399,269 in CDs & Vinyl (","[B0002N4JP2, 0760131694, B00150K8JC, B003MTXNV...","<img src=""https://images-na.ssl-images-amazon....",,,,0001526146,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,[],,[],,Very Best of White Heart,[],[https://images-na.ssl-images-amazon.com/image...,,Whiteheart,[],"929,975 in CDs &amp; Vinyl (",[],"<img src=""https://images-na.ssl-images-amazon....",,,$49.99,B000PWZRO2,{'ASIN:': 'B000PWZRO2'}
29996,[],,[1 - Intro\n2 - Best Of Me (Chronic 3000 remix...,,Dr. Dre &amp; Jay-Z: Chronic 3000,[],[https://images-na.ssl-images-amazon.com/image...,,DJ Eggnice,[],"692,178 in CDs &amp; Vinyl (",[],"<img src=""https://images-na.ssl-images-amazon....",,,,B000PX787U,{'ASIN:': 'B000PX787U'}
29997,[],,[1 \tBig Ol' Wren\n\t2 \tWho's Been Gettin' Th...,,Chairs,[],[https://images-na.ssl-images-amazon.com/image...,,Dave Landreth,[],"575,773 in CDs &amp; Vinyl (",[],"<img src=""https://images-na.ssl-images-amazon....",,,$9.99,B000PXYMC4,{'ASIN:': 'B000PXYMC4'}
29998,[],,[],,Under the Influence of,[],[https://images-na.ssl-images-amazon.com/image...,,Under the Influence of Giants,[],"868,526 in CDs &amp; Vinyl (",[],"<img src=""https://images-na.ssl-images-amazon....",,,$3.28,B000PY30C6,"{'Number of Discs:': '1', 'Label:': 'xxx', 'AS..."


In [20]:
top25 = pd.DataFrame({'ID': Recommend[0:25]})
top25

,ID
0,B000003CI2
1,B000026GO2
2,B00004BZ1H
3,B0000564WK
4,B00005LNPX
5,B000091NO4
6,B000254G94
7,B0002W1B22
8,B0006LINKC
9,B0007D08RA


In [23]:
Top25Recommend = top25.merge(data_set,left_on = 'ID', right_on = 'asin')
Top25Recommend[Top25Recommend.columns[:2]]
Top25Recommend = Top25Recommend[['ID','title']]
Top25Recommend.drop_duplicates().reset_index(drop=True)

,ID,title
0,B000003CI2,Mendelssohn (Sergei Krylov / Stefania Mormone)...
1,B000026GO2,Passages
2,B00004BZ1H,The Byron Sessions
3,B0000564WK,Walk Away Renee / Pretty Ballerina
4,B00005LNPX,Kamikaze
5,B000091NO4,Krush Groove
6,B000254G94,Cheap Thrills
7,B0002W1B22,Sciarrino: Un'immagine Di Arpocrate/6 Capricci...
8,B0006LINKC,EP Kate York
9,B0007D08RA,A Jazz Audio Biography
